**Tip**: Welcome to the Investigate a Dataset project! You will find tips in quoted sections like this to help organize your approach to your investigation. Before submitting your project, it will be a good idea to go back through your report and remove these sections to make the presentation of your work as tidy as possible. First things first, you might want to double-click this Markdown cell and change the title so that it reflects your dataset and investigation.

# Project: Investigate Carbon Per Capita


## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected for analysis. At the end of this section, describe the questions that you plan on exploring over the course of the report. Try to build your report around the analysis of at least one dependent variable and three independent variables.
>
> If you haven't yet selected and downloaded your data, make sure you do that first before coming back here. If you're not sure what questions to ask right now, then make sure you familiarize yourself with the variables and the dataset context for ideas of what to explore.

In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### General Properties

In [90]:
# read data
co2 = pd.read_csv("data/indicator CDIAC carbon_dioxide_emissions_per_capita.csv")

# how big is the dataset?
print co2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Columns: 254 entries, CO2 per capita to 2012
dtypes: float64(253), object(1)
memory usage: 466.4+ KB
None


In [91]:
# what does it look like?
print co2.head()

          CO2 per capita  1751  1755  1762  1763  1764  1765  1766  1767  \
0               Abkhazia   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1            Afghanistan   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2  Akrotiri and Dhekelia   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3                Albania   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4                Algeria   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

   1768    ...         2003      2004      2005      2006      2007      2008  \
0   NaN    ...          NaN       NaN       NaN       NaN       NaN       NaN   
1   NaN    ...     0.022704  0.027472  0.036780  0.047090  0.068312  0.131602   
2   NaN    ...          NaN       NaN       NaN       NaN       NaN       NaN   
3   NaN    ...     1.382066  1.332966  1.353789  1.224310  1.279420  1.297753   
4   NaN    ...     2.899236  2.762220  3.257010  3.113135  3.312875  3.328945   

       2009      2010      2011      2012  
0       NaN 

#### Observations

* The data goes back a long way, but there are a lot of NaNs in those early years.
* There are also NaNs in more recent years too, which could be informative.
* The country column heading should be changed.

In [92]:
# summary stats
co2.describe()

,1751,1755,1762,1763,1764,1765,1766,1767,1768,1769,...,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
count,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,200.000000,200.000000,200.000000,200.000000,201.000000,201.000000,201.000000,201.000000,65.000000,65.000000
mean,1.131253,1.105193,1.244749,1.239133,1.233543,1.227977,1.363424,1.357226,1.351055,1.344913,...,5.160597,5.266567,5.242741,5.250673,5.387444,5.293425,5.064836,5.189779,8.395975,8.391032
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.892710,7.222377,7.198735,6.950140,7.281876,6.858804,6.479270,6.538857,7.796528,8.041490
min,1.131253,1.105193,1.244749,1.239133,1.233543,1.227977,1.363424,1.357226,1.351055,1.344913,...,0.022704,0.027472,0.021237,0.025019,0.003482,0.008618,0.011942,0.008443,0.401268,0.426003
25%,1.131253,1.105193,1.244749,1.239133,1.233543,1.227977,1.363424,1.357226,1.351055,1.344913,...,0.610230,0.682390,0.730938,0.681245,0.645242,0.613104,0.609647,0.640119,4.196912,4.141014
50%,1.131253,1.105193,1.244749,1.239133,1.233543,1.227977,1.363424,1.357226,1.351055,1.344913,...,3.216404,3.242589,3.302545,3.301199,3.164370,3.277726,3.106366,3.263605,6.722385,6.506759
75%,1.131253,1.105193,1.244749,1.239133,1.233543,1.227977,1.363424,1.357226,1.351055,1.344913,...,7.514300,7.530032,7.296806,7.463160,7.555909,7.293089,6.782996,6.983513,9.513115,9.194220
max,1.131253,1.105193,1.244749,1.239133,1.233543,1.227977,1.363424,1.357226,1.351055,1.344913,...,55.322622,62.069377,63.187436,57.986895,57.066817,48.702062,41.378843,40.098333,41.220928,46.643197


In [93]:
# give better name to first columns
co2.rename(columns = {'CO2 per capita' : 'country'}, inplace = True)

In [94]:
co2.head()

,country,1751,1755,1762,1763,1764,1765,1766,1767,1768,...,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
0,Abkhazia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.022704,0.027472,0.036780,0.047090,0.068312,0.131602,0.213325,0.262174,NaN,NaN
2,Akrotiri and Dhekelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.382066,1.332966,1.353789,1.224310,1.279420,1.297753,1.215055,1.336544,NaN,NaN
4,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.899236,2.762220,3.257010,3.113135,3.312875,3.328945,3.564361,3.480977,3.562504,3.785654


In [95]:
# how many countries have data from 1751?
co2['1751'].describe()

count    1.000000
mean     1.131253
std           NaN
min      1.131253
25%      1.131253
50%      1.131253
75%      1.131253
max      1.131253
Name: 1751, dtype: float64

Not surprisingly, only one country has data going that far back. Which one?

In [96]:
# Which country has data for 1751?
co2.loc[pd.notnull(co2['1751'])]

,country,1751,1755,1762,1763,1764,1765,1766,1767,1768,...,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
219,United Kingdom,1.131253,1.105193,1.244749,1.239133,1.233543,1.227977,1.363424,1.357226,1.351055,...,9.069663,9.022278,9.001868,8.952901,8.684595,8.526467,7.705539,7.954469,7.324996,7.502533


But of course, the foremost colonial power of the era!

In [97]:
# How many countries do not have data in the last year?
missing_recent = co2.loc[pd.isnull(co2['2012'])]
len(missing_recent)

170

###  Sanity checks

Which countries have the highest cumulative CO2?


In [98]:
# change index for simpler aggregating
co2 = co2.set_index('country')

In [104]:
co2.sum(1).sort_values(ascending = False)

country
Qatar                       3005.390701
United States               2043.289976
Brunei                      2006.882965
United Kingdom              1784.762577
Luxembourg                  1758.837129
United Arab Emirates        1703.118653
Kuwait                      1624.240671
Bahrain                     1558.783103
Canada                      1410.204984
Belgium                     1379.276171
Trinidad and Tobago         1264.426704
Germany                     1210.906480
Australia                   1088.688431
Netherlands                  889.863537
Slovak Republic              845.036499
Denmark                      805.823307
Russia                       792.381253
Poland                       780.967275
France                       732.874690
New Caledonia                718.242462
Netherlands Antilles         706.418690
Austria                      697.931600
Saudi Arabia                 686.246180
Bahamas                      685.328774
South Africa                 643

#### Observations

Most of the top 10 are unsurprising: rich and/or oil-producing countries.

However, there are surprises too: Belgium, Trinidad & Tobago, and Luxembourg. Perhaps this is reflects data that goes back further for those countries. 

Let's compare with the means.

In [105]:
# which countries have the highest mean CO2?
co2.mean(1).sort_values(ascending = False)

country
Qatar                       46.959230
United Arab Emirates        31.539234
Netherlands Antilles        28.256748
Luxembourg                  26.649047
Brunei                      24.776333
Kuwait                      24.242398
Aruba                       20.710109
Bahrain                     19.984399
Nauru                       12.399117
Kazakhstan                  12.058241
Trinidad and Tobago         12.042159
New Caledonia               11.774467
Bahamas                     11.234898
United States                9.592911
Faeroe Islands               9.348407
Singapore                    8.978851
Greenland                    8.862988
Turkmenistan                 8.853965
Saudi Arabia                 8.798028
Palau                        8.313003
Slovenia                     7.587701
Estonia                      7.539403
Ukraine                      7.531082
Belgium                      7.496066
Cayman Islands               7.153721
Andorra                      7.106571
Unit

#### Observations

Now the top 10 is nearly all oil-rich countries. Nonetheless, we still have Luxembourg in the top 10, along with Netherlands Antilles. And other Caribbean island countries are near the top.

Also...the bottom results show that we have many rows without any values.

In [116]:
# how many rows without values?
co2.loc[co2.sum(1).isnull()]

,country,1751,1755,1762,1763,1764,1765,1766,1767,1768,...,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
0,Abkhazia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Akrotiri and Dhekelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,Channel Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,Cocos Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,East Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,Eritrea and Ethiopia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,Guam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,Guernsey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,Holy See,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Trim & add region data

CO2 patterns, I suspect, vary a lot by region and not just by country. 

For my analysis, I'd like to primarily focus on Europe since that's where I'm currently living. 

In [111]:
co2 = co2.reset_index()

In [112]:
co2.head()

,country,1751,1755,1762,1763,1764,1765,1766,1767,1768,...,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
0,Abkhazia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.022704,0.027472,0.036780,0.047090,0.068312,0.131602,0.213325,0.262174,NaN,NaN
2,Akrotiri and Dhekelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.382066,1.332966,1.353789,1.224310,1.279420,1.297753,1.215055,1.336544,NaN,NaN
4,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.899236,2.762220,3.257010,3.113135,3.312875,3.328945,3.564361,3.480977,3.562504,3.785654


In [113]:
def prep_df(df):
    # makes a copy & reshapes df so it can 
    # be converted to long (tidy) format
    df_copy = df.copy() 
    df_copy = df_copy.set_index('country')
    df_yr_ix = df_copy.T
    df_yr_ix = df_yr_ix.reset_index()
    df_yr_ix.rename(columns = {"index" : "year"}, inplace = True)
    
    # create list of country names for use as value columns
    cols = (list(df_yr_ix.columns.values))[1:]
    
    # convert to long format
    df_yr_long = pd.melt(df_yr_ix, id_vars = 'year', value_vars = cols)

    
    return df_yr_long

In [114]:
co2_long = prep_df(co2)

# verify by looking at country with data for all years
co2_long[co2_long.country == 'United Kingdom']

,year,country,value
55407,1751,United Kingdom,1.131253
55408,1755,United Kingdom,1.105193
55409,1762,United Kingdom,1.244749
55410,1763,United Kingdom,1.239133
55411,1764,United Kingdom,1.233543
55412,1765,United Kingdom,1.227977
55413,1766,United Kingdom,1.363424
55414,1767,United Kingdom,1.357226
55415,1768,United Kingdom,1.351055
55416,1769,United Kingdom,1.344913


In [ ]:
# transpose df

co2_my_countries = co2_my_countries.T

# reindex
co2_my_countries.reset_index(inplace=True)

# rename to year
co2_my_countries.rename(columns = {"index" : "year"}, inplace = True)


# create list of columns to use as value columns

cols = list(co2_my_countries.columns.values)[1:]
#cols = cols

# convert to long format

co2_my_countries_long = pd.melt(co2_my_countries, id_vars = 'year', value_vars = cols) 


In [ ]:
co2_my_countries_long.tail()

## Add region & sub-region columns

Data from http://www.internetworldstats.com/list1.htm

In [ ]:
# add continent and region info for
# better grouping

regions = pd.read_json('data/all_countries.json')
regions = regions[['name', 'region', 'sub-region']]

In [ ]:
regions.info()

In [ ]:
regions

Obvious problems:

* accents
* long-winded names (uk of gb & n ireland)
* parenthesis (virgin islands)
* extra spaces
* short form vs long form (us, uae)

In [ ]:
# give better name to country column
# also easier for merging with gapminder df

regions = regions.rename(columns = {'name':'country'})

In [ ]:
# how many countries are there?

pd.Series(test.country.unique())

In [ ]:
# how many different regionss?
regions.region.unique()

In [ ]:
# what are the None regions?
regions.loc[regions.region.isnull()]

In [ ]:
# merge co2 df with region df

co2_regions = test.merge(regions, how = 'left', on='country')

In [ ]:
co2_regions.groupby('country').mean().sort_values('value', ascending = False)

co2_regions[co2_regions.region == 'Europe']

In [ ]:
# plot

import seaborn as sns

sns.set(color_codes = True)
#sns.set_style({'axes.linewidth': '0.5'})


co2_regions.year = pd.Categorical(co2_regions.year, ordered = True)

In [ ]:
#p = sns.factorplot(x = 'year', y = 'value', hue = 'country', 
#               col = 'sub-region', sharey = False, 
#                   data = co2_regions[co2_regions.region == 'Europe'])

#p.set_xticklabels(rotation=45)
#p.set(ylim=(0,None))

In [ ]:
eur_subreg = co2_regions[co2_regions.region == 'Europe']['sub-region'].unique()


In [ ]:
co2_regions.loc[co2_regions['sub-region'].isin(eur_subreg)].country.unique()

In [ ]:
np.arange(int(co2_regions.year.min()), int(co2_regions.year.max()), 10.)

In [ ]:
eur_subreg

In [ ]:
co2_regions[co2_regions['sub-region'].isin(eur_subreg)].groupby(['sub-region', 'country']).sum()

In [ ]:
# get a better legend?

for subreg in eur_subreg:
    df = co2_regions[co2_regions['sub-region'] == subreg]
    fignum = pd.Index(eur_subreg).get_loc(subreg)
    plt.figure(fignum)
    
    ax = sns.pointplot(x = 'year', y = 'value', hue = 'country', 
                   col = subreg, data = df, scale = .5)
    ax.legend(loc = 0)
    ax.set(ylim=(0,None))
    ax.set(xlim=(1950,None))
    #plt.xticks(np.arange(int(co2_regions.year.min()), int(co2_regions.year.max()), 10.))
    #ax.set_xticklabels(rotation=45)
    plt.title(subreg)
    

In [ ]:
# which countries had no corresponding regional info
co2_totals = co2_regions[co2_regions.region.isnull()].groupby('country').sum()
co2_totals

In [ ]:
# Do countries with NaN aggregate values have any values?

all_nulls = set(co2_totals[co2_totals.value.isnull()].index.values)


In [ ]:
# all_nulls
co2[co2.country.isin(all_nulls)]

In [ ]:
#co2_regions.head()
missing_region = set(co2_regions[co2_regions.region.isnull()].country.unique())
no_reg_no_vals = missing_region.intersection(all_nulls)
list(no_reg_no_vals)

In [ ]:
# countries missing regions but with co2 values
no_reg_with_vals = co2_regions[co2_regions.country.isin(missing_region)]
no_reg_with_vals = no_reg_with_vals[no_reg_with_vals.value.notnull()].country.unique()
no_reg_with_vals

In [ ]:
country_lists = pd.Series(no_reg_with_vals).apply(str.split)

In [ ]:
regions.country.head()

In [ ]:
# remove accents from countries in regions df
import unidecode
regions.country = regions.country.apply(unidecode.unidecode)
regions.country.head()

In [ ]:
import pprint as pp
matches = []
for l in country_lists:
    if regions.country.str.startswith(l[0]).any():
        match = {regions[regions.country.str.startswith(l[0])].country.values[0]: " ".join(l)}
        matches.append(match)
pp.pprint(matches)

In [ ]:
# manually searched for correct match for 'British Indian Ocean Territory':
co2_regions[co2_regions.country.str.contains('Ocean')].country.unique()

# no match found
matches.pop(1)
pp.pprint(matches)

In [ ]:
# look for matches to 'Netherlands': 'Netherlands Antilles'
regions[regions.country.str.contains('Curacao')].country
matches.append({'Curacao': matches[6].values()[0]})

pp.pprint(matches)


In [ ]:
# remove wrong entry for Netherlands antilles
matches.pop(6)
pp.pprint(matches)

In [ ]:
# fix the 'united' countries
print "From co2_regions:"
print co2_regions[co2_regions.country.str.startswith('United')].country.unique()
print 
print "From regions:"
(regions[regions.country.str.startswith('United')].country.values)

In [ ]:
matches.append({'United States of America': 'United States'})
matches.append({'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom'})
pp.pprint(matches)

In [ ]:
# remove UAE (it's ok in merged df)
matches.pop(12)
pp.pprint(matches)

In [ ]:
# find match for 'West Bank and Gaza'

regions[regions.country.str.contains('Palestin')].country

In [ ]:
len(matches)
matches.pop(14)

In [ ]:
matches.append({'Palestine, State of': 'West Bank and Gaza'})
pp.pprint(matches)

In [ ]:
for l in matches:
    print l.keys()[0]

In [ ]:
%timeit
for d in matches:
    print d.values()[0]
    #print d.keys()
    r = regions.loc[regions.country == d.keys()[0]].region.item()
    sr = regions.loc[regions.country == d.keys()[0]]['sub-region'].item()
    #print r
    #print sr
    co2_regions.loc[co2_regions.country == d.values()[0], 'region'] = r
    co2_regions.loc[co2_regions.country == d.values()[0], 'sub-region'] = sr
    
    #print regions[regions.country == d.keys()]
    #co2_regions[co2_regions.country == d.values()].region = regions[regions.country == d.keys()].region
    #co2_regions[co2_regions.country == d.values()].sub-region = regions[regions.country == d.keys()].sub-region
    #print d.keys()
    #print d.values()

In [ ]:
# how many countries are still missing regions?
missing_region = set(co2_regions[co2_regions.region.isnull()].country.unique())


> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning (Replace this with more specific notes!)

In [ ]:
# which countries do not have a match in regions?
countries = list(co2[co2.columns[0]])
regions_list = list(regions['name'].values)

In [ ]:
no_region = [i for i in countries if i not in regions_list]
len(no_region)

In [ ]:
no_region

In [ ]:
# and vice-versa

no_country = [i for i in regions_list if i not in countries]
len(no_country)

In [ ]:
no_country

In [ ]:
# how many more matches did we get?
print "Before:"
print len(no_country)
print ""
print "After:"

regions_list = list(regions['name'].values)
no_country = [i for i in regions_list if i not in countries]
len(no_country)


In [ ]:
# more cleanup

bvi = 'British Virgin Islands'
vib = 'Virgin Islands (British)'

bvi = bvi.split()
vib = vib.split()
vib_clean = [i.strip("()") for i in vib]

for word in bvi:
    if word in vib_clean:
        print "match"
    else:
        print "no match"



In [ ]:
# TODO: remove words in ()
# str.replace(r"\(.*\)","")

vib = 'Virgin Islands (British)'
import re
#print re.sub(r'[^A-Za-z0-9 ]+', '', vib)
print re.sub('\W+',' ', vib )


#vib.replace("(","")

In [ ]:
test = ['South', 'Georgia', 'the', 'South', 'Sandwich', 'Islands']
for i in test: 
    if i in connector_words:
        test.remove(i)
        print test
    else:
        pass

In [ ]:
def remove_special_char(s):
    clean_s = re.sub('\W+',' ', s)
    return clean_s
    

names_cleaned = regions['name'].apply(remove_special_char)
names_cleaned_list = names_cleaned.str.split()
connector_words = ['and', 'et', 'of', 'the']

for l in names_cleaned_list:
    new_list = []
    for word in l:
        if word in connector_words:
            l.remove(word)
        else:
            pass
    new_list.append(l)
    print l
       


In [ ]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
import seaborn as sns

sns.set(color_codes = True)
#sns.set_style({'axes.linewidth': '0.5'})


co2_my_countries_long.year = pd.Categorical(co2_my_countries_long.year, ordered = True)



p = sns.factorplot(x = 'year', y = 'value', hue = 'country', 
               row = 'region', size = 6, data = co2_my_countries_long)

p.set_xticklabels(rotation=45)
p.set(ylim=(0,None))


co2_my_countries_long.groupby(['sub-region', 'country']).describe()

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!